In [ ]:
class RedditPost:
    def __init__(self, data):

        useful_attributes=('post_hint','subreddit','ups','title','score','url','num_comments','name','over_18','downs','is_self')

        #setting attributes
        for key in useful_attributes:
            setattr(self, key, data[key])
        pass

    def download(self):

        if self.post_hint=="image" and self.is_self==False:
            response = requests.get(self.url, stream=True)
            
            #parsing title for image name
            title_=re.sub(r"[\s\]0-9\[]", '', self.title)
            
            #splitting extension and path 
            path,extension=os.path.splitext(self.url)
            subbreddit=self.subreddit
            filename="wallpaperss/{}/{}{}".format(subbreddit,title_,extension)
            
            #making directory according to subbreddit
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            file = open(filename, "wb")
            file.write(response.content)
            file.close()
            return file
            pass
        pass




    def __str__(self):
        return "RedditPost(({} {} ): Url {})".format( self.title,self.score, self.url)


def query(subreddit,score):


    no_of_post=0
    try:
        ress=requests.get("https://www.reddit.com/r/{}/.json".format(subreddit),headers={'User-Agent': 'hanphir'})
        #ress.raise_for_status()


        ress.raise_for_status()
        jdata=ress.json()
        post=jdata["data"]["children"]
        for i in range(len(post)):
            r=post[i]["data"].get("score")
            if r > score:
                no_of_post+=1
                pass
            pass
        return post,no_of_post

    except requests.exceptions.HTTPError as e:
        print("not a valid subreddit or forbidden")
    except requests.exceptions.ConnectionError as e:
        print("not connected to the internet")
    except requests.exceptions.InvalidURL as e:
        print("please enter a valid url")
        pass
    pass


def main():
    # d = dict && s=score, download wallpapers from funny+spaceporn whose score is greater than 500
    d,s=query("funny+spaceporn",500)

    #creating a single object 
    f=RedditPost(d[11]["data"])
   



    #converting into list of RedditPost Objects
    li=[RedditPost(d[i]["data"])for i in range(len(d))]
    
    #downloading all wallpapers into directory wallpaperss/{your subbreddit name }{title of image}.{extension of your image}
    for o in li:
        o.download()




    pass

if __name__ == '__main__':


    main()